# Zadanie domowe
Napisać program sprowadzający wyrażenie logiczne do najprostszej postaci.

Wyrażenie może zawierać:
- zmienne logiczne (ciąg znaków alfanumerycznych)
- operatory: koniunkcji, alternatywy, alternatywy wykluczającej, negacji, implikacji i równoważności
- stałe logiczne (0 lub 1)
- spacje

Wyjściem programu ma być wyrażenie logiczne równoważne danemu

**Czas oddania: północ przed 4. zajęciami laboratoryjnymi**

In [30]:
from operator import or_, and_, not_, xor, eq
from functools import reduce
from itertools import combinations, chain

def imp(a, b):
    return not(a) or b


import string
VARS = string.ascii_lowercase
VALUES = "01"

OPS = "|&>~^="

FUN_OPS = {"|": or_, "&": and_, ">": imp, "~": not_, "^": xor, "=": eq  }

PREC = {'~': 6, 
        '&': 4, 
        '|': 3, 
        '^': 5, 
        '>': 2, 
        '=': 1}

ASSO = {'&': 'right',
        '=': 'right', 
        '>': 'right',
        '^': 'right',
        '|': 'right',
        '~': 'left'}


In [82]:
#TODO 
# full word variable names
# little bit of opitmization of calculations
# always true/false expressions
# comments


def check_expr(expr):
    expr = "".join(expr.split())
    prev_token = expr[0]
    par_count = 0
    
    for token in expr[1:]:
        
        if prev_token in OPS:
            if token not in VARS and token not in VALUES and token != '(':
                return False
            
        if prev_token in VARS or prev_token in VALUES:
            if token not in OPS and token != ')':
                return False
        
        if prev_token == ')':
            if token not in OPS and token != ')':
                return False
            
            par_count -= 1
            
        if prev_token == '(':
            if token not in VARS and token not in VALUES and token != '(' and token != '~':
                return False
            
            par_count += 1
            
            
        if(par_count < 0):
            return False
        
        prev_token = token
        
    if prev_token == '(':
        par_count += 1
    
    if prev_token == ')':
        par_count -= 1
        
    if par_count != 0:
            return False
        
        
    return True


def get_tokens(expr):
    tokens = set(list("".join(expr.split())))
    
    operators = tokens  & set(VARS)
    
    variables = tokens & set(OPS)
    
    return sorted(operators), sorted(variables)


def convert_to_rpn(expr):
    output = ""
    op_stack = []

    for token in expr:

        if token in VARS or token in VALUES:
            output += token

        if token in OPS:

            if op_stack and op_stack[-1] != "(":
                curr_prec = PREC[token]

                top_asso = ASSO[op_stack[-1]]
                top_prec = PREC[op_stack[-1]]

                while (op_stack and op_stack[-1] != "(" and
                       top_prec > curr_prec or 
                       (top_prec == curr_prec and top_asso == 'left')
                      ):

                    output += op_stack.pop()

                    if op_stack and op_stack[-1] != "(":
                        top_prec = PREC[op_stack[-1]]
                        top_asso = ASSO[op_stack[-1]]

            op_stack.append(token)

        if token == "(":
            op_stack.append(token)

        if token == ")":
            while op_stack[-1] != "(" :
                output += op_stack.pop()

            op_stack.pop()

    while op_stack:
        output += op_stack.pop()

    return output


def fill_zeros(bin_num, res_digits):
    
    to_fill = res_digits - len(bin_num)
    
    return "".join([str('0') for _ in range(to_fill)]) + bin_num;


def generate_values(variables):
    
    max_value = len(variables)
    
    base_values = [bin(i)[2:] for i in range(2**max_value)]
    
    max_digits = len(base_values[-1])
    
    filled_values = map(lambda i: fill_zeros(i, max_digits), base_values) 
    
    separated_values = list(map(list, filled_values))
    
    values = list(map(lambda v: dict(zip(variables, v)), separated_values))
    
    bool_infos = [{'var_values': x, 'result': None} for x in values]
    
    return bool_infos


def substitute_values(rpn_expr, bool_info):
    valued_rpn_expr = ""
        
    for token in rpn_expr:
            
        to_add = token

        if token in VARS:
            to_add = bool_info['var_values'][token]

        valued_rpn_expr += to_add
            
    return valued_rpn_expr


def evaluate(rpn_expr, bool_info):
    
    valued_rpn_expr = substitute_values(rpn_expr, bool_info)
    
    str_to_bool = lambda s: True if s == "1" else False
    bool_to_str = lambda b: "1" if b else "0"
    
    stack = []
    
    for token in valued_rpn_expr:
            
        if token == "~":
            var = str_to_bool(stack.pop())

            res = bool_to_str(FUN_OPS["~"](var))

        elif token in OPS and token != "~":
            var2 = str_to_bool(stack.pop())

            var1 = str_to_bool(stack.pop())

            res = bool_to_str(FUN_OPS[token](var1, var2))

        else:
            res = token

        stack.append(res)
        
    bool_info['result'] = stack.pop()
    


def evaluate_all(expr):
    
    rpn_expr = convert_to_rpn(expr)
    
    variables, operators = get_tokens(rpn_expr)
    
    bool_infos = generate_values(variables)
    
    for bool_info in bool_infos:
        
        evaluate(rpn_expr, bool_info)
        
    return bool_infos


#MACKLUSKY STARTS HERE
#bool_info_example = 
# {
#     'var_values': {'a': '0', 'b': '1', 'c': '0'}
#     'result': '0'
# }

#qm_info_example = 
# {
#     'var_values': {'a': '0', 'b': '1', 'c': '-'}
#     'properties': {'num': [2, 3], 'ones': 1, 'checked': True}
# }

#qm_group =
# [ qm_info1, qm_info2, qm_info_3 ] #all with the same number of ones

#qm_cluster = 
# {
#     0: [qm_info1, qm_info2, qm_info_3]  #qm_group0
#     1: [qm_info4, qm_info5]             #qm_group1
#     2: []                               #qm_group2
#     3: [qm_info5]                       #qm_group3
# }


def sum_of_ones(bool_info):
    
    var_values = bool_info['var_values'].values()
    
    ones = reduce(lambda x, y: (x + 1) if y == '1' else x, var_values, 0)
    
    return ones


def convert_to_qm_info(bool_info):
    qm_info = {}
    qm_info['var_values'] = bool_info['var_values'].copy()
    qm_info['properties'] = {}
    
    bin_no = ""
    for var_value in bool_info['var_values'].values():
        bin_no += var_value
        
    qm_info['properties']['no'] = [int(bin_no, 2)]
    qm_info['properties']['checked'] = False
    qm_info['properties']['ones'] = sum_of_ones(bool_info)
    
    return qm_info
    

def get_positive_qm_infos(bool_infos):
    positive_bool_infos = filter(lambda x: x['result'] == '1', bool_infos)
    
    positive_qm_infos = [convert_to_qm_info(i) for i in positive_bool_infos]
    
    return positive_qm_infos
    

def make_initial_cluster(qm_infos):
    
    var_count = len(qm_infos[0]['var_values'])
    
    cluster = {i:[] for i in range(var_count + 1)}
        
    for qm_info in qm_infos:
        cluster[qm_info['properties']['ones']].append(qm_info) 
    
    return cluster


def compare_qm_infos(qm_info1, qm_info2):
    
    diff_var_values = set(qm_info1['var_values'].items()) - set(qm_info2['var_values'].items())
    
    diff_key = list(diff_var_values)[0][0] if len(diff_var_values) == 1 else None
    
    return diff_key


def exist_in_group(new_no, group):
    group_nos = [qm_info['properties']['no'] for qm_info in group]
    
    return new_no in group_nos


def make_next_cluster(prev_cluster):
    
    for ones in prev_cluster.keys():
        if prev_cluster[ones]:
            var_count = len(prev_cluster[ones][0]['var_values'])
    
    next_cluster = {i:[] for i in range(var_count + 1)}
    
    for ones, group in prev_cluster.items():
        
        for qm_info1 in group:
            
            next_ones = ones + 1
            
            if next_ones in prev_cluster.keys():
                
                next_group = prev_cluster[next_ones]
                
                for qm_info2 in next_group:
                    
                    diff_key = compare_qm_infos(qm_info1, qm_info2)
                    
                    if diff_key:
                        
                        new_ones = qm_info1['properties']['ones']
                        
                        new_group = next_cluster[new_ones]
                        
                        new_no = [no for no in qm_info1['properties']['no'] + qm_info2['properties']['no']]
                        
                        new_no = sorted(new_no)                    
                        
                        if not exist_in_group(new_no, new_group):
                            
                            new_qm_info = {}
                            new_qm_info['var_values'] = dict(qm_info1['var_values'].copy())
                            new_qm_info['var_values'][diff_key] = '-'
                            new_qm_info['properties'] = {}
                            new_qm_info['properties']['no'] = new_no
                            new_qm_info['properties']['checked'] = False
                            new_qm_info['properties']['ones'] = qm_info1['properties']['ones']
                            new_group.append(new_qm_info)
                            
                        qm_info1['properties']['checked'] = True
                        qm_info2['properties']['checked'] = True
    
    return next_cluster


def group_clusters(bool_infos):
    qm_infos = get_positive_qm_infos(bool_infos)
    
    state = True
    clusters = [make_initial_cluster(qm_infos)]

    while state:
        clusters.append(make_next_cluster(clusters[-1]))
        state = reduce(lambda x, y: bool(x or y), list(clusters[-1].values()), False)
        
    return clusters


def filter_unchecked_qm_infos(clusters):

    unchecked_qm_infos = []

    for cluster in clusters:
        for group in cluster.values():
            for qm_info in group:
                if qm_info['properties']['checked'] == False:
                    unchecked_qm_infos.append(qm_info)
                    
    return unchecked_qm_infos


def choose_min_qm_infos(unchecked_qm_infos):
    
    positive_nos = set([])

    for qm_info in unchecked_qm_infos:
        positive_nos = positive_nos | set(qm_info['properties']['no'])
    
    
    positive_nos = list(positive_nos)
    
    unchecked_nos = [qm_info['properties']['no'] for qm_info in unchecked_qm_infos]
    
    for combination_length in range(len(unchecked_nos) + 1):
        
        qm_combinations = combinations(unchecked_nos, combination_length)
        
        for qm_combination in qm_combinations:
            
            unnested_qm_combination = list(chain.from_iterable(qm_combination))
            
            if set(unnested_qm_combination) == set(positive_nos):
                return [i for i in unchecked_qm_infos if i['properties']['no'] in qm_combination]

            
def qm_infos_to_string(chosen_qm_infos):

    shortened_expr = ""
    
    chosen_var_values = [i['var_values'] for i in chosen_qm_infos]

    for var_value in chosen_var_values:
        for var in var_value.keys():
            to_add = ""
            
            if var_value[var] == '0':
                to_add = '~' + var + '&'
            if var_value[var] == '1':
                to_add = var + '&'
                
            shortened_expr += to_add
                
        if shortened_expr[-1] == '&':
            shortened_expr = shortened_expr[:-1]
            
        shortened_expr += ' | '

    if shortened_expr[-3:] == ' | ':
            shortened_expr = shortened_expr[:-3]

    return shortened_expr


def shorten_expr(expr):
    if not check_expr(expr):
        print("Invalid expression!");
        return
    
    variables, operators = get_tokens(expr)
    
    bool_infos = evaluate_all(expr)
    
    clusters = group_clusters(bool_infos)
    
    unchecked_qm_infos = filter_unchecked_qm_infos(clusters)
    
    chosen_qm_infos = choose_min_qm_infos(unchecked_qm_infos)
    
    return qm_infos_to_string(unchecked_qm_infos)

In [75]:
expr = "(a & ((b | c & d) ))"

In [76]:
check_expr(expr)

True

In [77]:
convert_to_rpn(expr)

'abcd&|&'

In [78]:
short_expr = shorten_expr(expr)

In [79]:
short_expr

'a&c&d | a&b'

In [80]:
evaluate_all(expr) == evaluate_all(short_expr)

True

In [81]:
evaluate_all(expr)

[{'result': '0', 'var_values': {'a': '0', 'b': '0', 'c': '0', 'd': '0'}},
 {'result': '0', 'var_values': {'a': '0', 'b': '0', 'c': '0', 'd': '1'}},
 {'result': '0', 'var_values': {'a': '0', 'b': '0', 'c': '1', 'd': '0'}},
 {'result': '0', 'var_values': {'a': '0', 'b': '0', 'c': '1', 'd': '1'}},
 {'result': '0', 'var_values': {'a': '0', 'b': '1', 'c': '0', 'd': '0'}},
 {'result': '0', 'var_values': {'a': '0', 'b': '1', 'c': '0', 'd': '1'}},
 {'result': '0', 'var_values': {'a': '0', 'b': '1', 'c': '1', 'd': '0'}},
 {'result': '0', 'var_values': {'a': '0', 'b': '1', 'c': '1', 'd': '1'}},
 {'result': '0', 'var_values': {'a': '1', 'b': '0', 'c': '0', 'd': '0'}},
 {'result': '0', 'var_values': {'a': '1', 'b': '0', 'c': '0', 'd': '1'}},
 {'result': '0', 'var_values': {'a': '1', 'b': '0', 'c': '1', 'd': '0'}},
 {'result': '1', 'var_values': {'a': '1', 'b': '0', 'c': '1', 'd': '1'}},
 {'result': '1', 'var_values': {'a': '1', 'b': '1', 'c': '0', 'd': '0'}},
 {'result': '1', 'var_values': {'a': '

In [74]:
evaluate_all(short_expr)

[{'result': '0', 'var_values': {'a': '0', 'b': '0'}},
 {'result': '0', 'var_values': {'a': '0', 'b': '1'}},
 {'result': '0', 'var_values': {'a': '1', 'b': '0'}},
 {'result': '1', 'var_values': {'a': '1', 'b': '1'}}]